# Analysis of fingerprint segmentation results

In [1]:
import tensorflow as tf
import cv2 as cv
from utils import show
from ipywidgets import interact
from IPython.display import clear_output
import json

import os, sys
sys.path.append(os.path.dirname(os.path.realpath("."))) # add parent path to import the pyfing package of this project

import pyfing as pf

In [2]:
PATH_FVC = '../../datasets/'
PATH_GT = '../../datasets/segmentationbenchmark/groundtruth/'
PATH_RES = '../../results/'
PATH_PARAMS = '../parameters/segmentation/'

In [4]:
alg = pf.Gmfs()
#alg = pf.Sufs(models_folder="../models/")
results = None
year = 2000
db = 1
subset = 'a'

In [7]:
@interact(y=[2000,2002,2004], n=[1,2,3,4])
def choose_db(y=year, n=db):
    global results, year, db
    year, db = y, n
    with open(f'{PATH_RES}fvc{year}_db{db}_{subset}_{type(alg).__name__}_res.txt', 'r') as filehandle:
        results = json.load(filehandle)

    results = sorted(results, key=lambda r: r[2][0])
    alg.parameters = alg.parameters.load(f'{PATH_PARAMS}fvc{year}_db{db}_b_{type(alg).__name__}_params.json')


interactive(children=(Dropdown(description='y', options=(2000, 2002, 2004), value=2000), Dropdown(description=…

In [17]:
@interact(i=(0, len(results)-1))
def show_result(i=0):
    i, j, (er, dc, jc) = results[i]
    img = cv.imread(f'{PATH_FVC}fvc{year}/db{db}_{subset}/{i}_{j}.png', cv.IMREAD_GRAYSCALE)
    gt = cv.bitwise_not(cv.imread(f'{PATH_GT}fvc{year}_db{db}_im_{i}_{j}seg.png', cv.IMREAD_GRAYSCALE))
    mask = alg.run(img)
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    gt_contours, _ = cv.findContours(gt, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)    
    img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    img_res = cv.drawContours(img_bgr.copy(), contours, -1, (255,0,0), 5)
    img_gt = cv.drawContours(img_bgr.copy(), gt_contours, -1, (0,192,0), 3)
    img_err = img_bgr.copy()
    false_positives = cv.bitwise_and(mask, cv.bitwise_not(gt))
    false_negatives = cv.bitwise_and(cv.bitwise_not(mask), gt)
    img_err[false_positives == 255,1:3] //= 2
    img_err[false_negatives == 255,0:3:2] //= 2
    cv.drawContours(img_err, gt_contours, -1, (0,192,0), 3)
    cv.drawContours(img_err, contours, -1, (255,0,0), 5)
    show((img_res, f'{i}_{j}'), (img_gt, 'Ground truth'), (img_err, f'ER = {er:.2f}% DC = {dc:.2f} JC = {jc:.2f}'))

interactive(children=(IntSlider(value=0, description='i', max=799), Output()), _dom_classes=('widget-interact'…